In [1]:
import Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from datetime import date

/Users/calumthompson/Google Drive/Fantasy_football/Setup.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Load most recent player_GWs

In [2]:
list_of_input_files = glob.glob(Setup.input_data_folder + '*')
latest_input_address = max(list_of_input_files, key=os.path.getctime)
print('Opening ', latest_input_address)

player_GWs = pd.read_csv(latest_input_address)

Opening  data/input_data/2020-12-03.csv


# Formulate KPIs

In [3]:
player_KPIs = player_GWs.loc[player_GWs['FIX_season'] == Setup.this_season].copy()

player_KPIs['LW_total_points'] = player_KPIs.groupby(['second_name','first_name'])['total_points'].shift(periods = 1 )
player_KPIs['L4W_mean_points'] = player_KPIs.groupby(['second_name','first_name'])['LW_total_points'].rolling(4, min_periods = 1).mean().reset_index(level=['second_name','first_name'], drop = True)
player_KPIs['L8W_mean_points'] = player_KPIs.groupby(['second_name','first_name'])['LW_total_points'].rolling(8, min_periods = 1).mean().reset_index(level=['second_name','first_name'], drop = True)


temp = player_KPIs.loc[player_KPIs['forecast'] == 1].groupby(['second_name','first_name','position','FIX_team','cost_today']).first()[['LW_total_points','L4W_mean_points', 'L8W_mean_points','forecast_points']]
temp['N8W_mean_forecast'] = player_KPIs.loc[player_KPIs['forecast'] == 1].groupby(['second_name','first_name','position','FIX_team','cost_today']).head(8).groupby(['second_name','first_name','position','FIX_team'])['forecast_points'].mean()

player_KPIs = temp.reset_index().rename(columns = {'L4W_total_points' : 'L4W_mean_points',
                                                          'L8W_total_points' : 'L8W_mean_points',
                                                          'forecast_points'  : 'NW_forecast'})


player_KPIs['cost_today'] = player_KPIs['cost_today']/ 10
player_KPIs['performance_per_cost'] = player_KPIs['L8W_mean_points'] / player_KPIs['cost_today']
player_KPIs['forecast_per_cost'] = player_KPIs['N8W_mean_forecast'] / player_KPIs['cost_today']

player_KPIs = player_KPIs[['second_name',
                         'first_name',
                         'position',
                         'FIX_team',
                         'cost_today',
                         'LW_total_points',
                         'L4W_mean_points',
                         'L8W_mean_points',
                         'performance_per_cost',
                         'NW_forecast',
                         'N8W_mean_forecast',
                         
                         
                         'forecast_per_cost']].sort_values('N8W_mean_forecast', ascending = False)


# Load most recent team 

In [4]:
def load_most_recent_team():
    list_of_team_addresses = glob.glob(Setup.teams_folder + '*')
    latest_team_address = max(list_of_team_addresses, key=os.path.getctime)
    print('Opening ', latest_team_address)

    return pd.read_csv(latest_team_address)

current_team = load_most_recent_team()

# current_team

Opening  data/teams/2020-11-25.csv


## Join to GW data

In [5]:
team_GWs = pd.merge(current_team,player_GWs)

## Join to KPI data

In [6]:
team_KPIs = pd.merge(current_team,player_KPIs)
player_KPIs['in_team'] = np.where(player_KPIs['second_name'].isin(team_KPIs['second_name']), 1 ,0)

# Looking forward

## Next GW

In [7]:
team_KPIs.sort_values('NW_forecast', ascending = False)

,first_name,second_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost
12,Dominic,Calvert-Lewin,4.0,Everton,7.9,2.0,7.00,5.875,0.743671,6.700556,6.517554,0.825007
8,Heung-Min,Son,3.0,Spurs,9.5,3.0,4.50,7.625,0.802632,6.441579,6.353211,0.668759
7,Jack,Grealish,3.0,Aston Villa,7.7,10.0,8.00,8.500,1.103896,6.387750,6.160526,0.800068
11,Harry,Kane,4.0,Spurs,10.9,2.0,5.25,7.875,0.722477,6.255753,6.240253,0.572500
6,Kurt,Zouma,2.0,Chelsea,5.5,5.0,6.75,6.250,1.136364,5.508967,5.299313,0.963511
13,Timo,Werner,4.0,Chelsea,9.5,2.0,5.25,5.375,0.565789,4.596739,4.365078,0.459482
10,Hakim,Ziyech,3.0,Chelsea,8.3,2.0,7.50,4.000,0.481928,4.450288,4.154481,0.500540
0,Emiliano,MartÃ­nez,1.0,Aston Villa,4.8,1.0,2.00,3.625,0.755208,3.985748,3.143807,0.654960
4,Tyrone,Mings,2.0,Aston Villa,5.3,1.0,3.50,4.250,0.801887,3.955005,3.299264,0.622503
1,Alex,McCarthy,1.0,Southampton,4.6,2.0,4.50,5.000,1.086957,3.265489,2.949522,0.641201


## Next 8GW

In [8]:
team_KPIs.sort_values('N8W_mean_forecast', ascending = False)

,first_name,second_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost
12,Dominic,Calvert-Lewin,4.0,Everton,7.9,2.0,7.00,5.875,0.743671,6.700556,6.517554,0.825007
8,Heung-Min,Son,3.0,Spurs,9.5,3.0,4.50,7.625,0.802632,6.441579,6.353211,0.668759
11,Harry,Kane,4.0,Spurs,10.9,2.0,5.25,7.875,0.722477,6.255753,6.240253,0.572500
7,Jack,Grealish,3.0,Aston Villa,7.7,10.0,8.00,8.500,1.103896,6.387750,6.160526,0.800068
6,Kurt,Zouma,2.0,Chelsea,5.5,5.0,6.75,6.250,1.136364,5.508967,5.299313,0.963511
13,Timo,Werner,4.0,Chelsea,9.5,2.0,5.25,5.375,0.565789,4.596739,4.365078,0.459482
10,Hakim,Ziyech,3.0,Chelsea,8.3,2.0,7.50,4.000,0.481928,4.450288,4.154481,0.500540
4,Tyrone,Mings,2.0,Aston Villa,5.3,1.0,3.50,4.250,0.801887,3.955005,3.299264,0.622503
0,Emiliano,MartÃ­nez,1.0,Aston Villa,4.8,1.0,2.00,3.625,0.755208,3.985748,3.143807,0.654960
1,Alex,McCarthy,1.0,Southampton,4.6,2.0,4.50,5.000,1.086957,3.265489,2.949522,0.641201


# Transfers

## Best looking players 

In [9]:
for i in range(1,5):
    print('position ', i)
    display(player_KPIs.loc[player_KPIs['position'] == i].head(20))
    print()

position  1


,second_name,first_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost,in_team
161,Fabianski,Lukasz,1.0,West Ham,5.1,3.0,7.25,6.375000,1.250000,5.080901,5.324038,1.043929,0
300,Lloris,Hugo,1.0,Spurs,5.5,10.0,7.00,5.250000,0.954545,4.080624,3.886536,0.706643,0
352,Meslier,Illan,1.0,Leeds,4.5,11.0,4.75,5.000000,1.111111,3.248677,3.842630,0.853918,0
466,Santana de Moraes,Ederson,1.0,Man City,6.0,6.0,4.00,3.625000,0.604167,4.149465,3.664445,0.610741,0
124,Darlow,Karl,1.0,Newcastle,5.0,8.0,3.50,4.375000,0.875000,3.467562,3.466849,0.693370,0
294,Leno,Bernd,1.0,Arsenal,5.0,2.0,5.25,3.875000,0.775000,2.482755,3.371253,0.674251,0
425,Ramsdale,Aaron,1.0,Sheffield Utd,4.8,3.0,2.75,2.750000,0.572917,2.970533,3.363073,0.700640,0
349,Mendy,Edouard,1.0,Chelsea,5.2,6.0,5.00,4.714286,0.906593,3.965755,3.331416,0.640657,0
595,dos Santos PatrÃ­cio,Rui Pedro,1.0,Wolves,5.5,2.0,5.00,4.375000,0.795455,1.940149,3.213018,0.584185,0
255,Johnstone,Sam,1.0,West Brom,4.5,11.0,4.75,4.875000,1.083333,3.600538,3.182846,0.707299,0



position  2


,second_name,first_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost,in_team
105,Chilwell,Benjamin,2.0,Chelsea,6.2,8.0,7.25,7.000000,1.129032,6.077654,5.879003,0.948226,0
249,James,Reece,2.0,Chelsea,5.1,6.0,5.50,4.000000,0.784314,6.072959,5.744894,1.126450,0
590,Zouma,Kurt,2.0,Chelsea,5.5,5.0,6.75,6.250000,1.136364,5.508967,5.299313,0.963511,1
439,Robertson,Andrew,2.0,Liverpool,7.2,2.0,4.50,4.375000,0.607639,4.461318,4.634664,0.643703,0
391,Ogbonna,Angelo,2.0,West Ham,4.9,9.0,5.25,4.375000,0.892857,3.477742,4.286005,0.874695,0
121,Dallas,Stuart,2.0,Leeds,4.5,8.0,4.75,4.250000,0.944444,3.778163,4.278778,0.950840,0
515,Targett,Matt,2.0,Aston Villa,4.5,1.0,3.00,3.125000,0.694444,4.650828,4.016634,0.892585,0
277,Konsa Ngoyo,Ezri,2.0,Aston Villa,4.7,1.0,3.75,3.625000,0.771277,4.419802,3.655120,0.777685,0
433,ReguilÃ³n,Sergio,2.0,Spurs,5.6,5.0,5.25,3.857143,0.688776,3.842983,3.543533,0.632774,0
120,Cresswell,Aaron,2.0,West Ham,5.2,2.0,4.00,5.625000,1.081731,3.181768,3.538392,0.680460,0



position  3


,second_name,first_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost,in_team
498,Son,Heung-Min,3.0,Spurs,9.5,3.0,4.50,7.625,0.802632,6.441579,6.353211,0.668759,1
461,Salah,Mohamed,3.0,Liverpool,12.2,6.0,5.75,6.250,0.512295,6.283781,6.330247,0.518873,0
202,Grealish,Jack,3.0,Aston Villa,7.7,10.0,8.00,8.500,1.103896,6.387750,6.160526,0.800068,1
69,Borges Fernandes,Bruno Miguel,3.0,Man Utd,10.9,10.0,10.00,9.000,0.825688,5.872427,6.041515,0.554267,0
320,Mahrez,Riyad,3.0,Man City,8.3,21.0,6.50,5.625,0.677711,6.225814,5.852693,0.705144,0
429,Rashford,Marcus,3.0,Man Utd,9.5,5.0,3.75,5.125,0.539474,4.695017,4.804263,0.505712,0
32,Aubameyang,Pierre-Emerick,3.0,Arsenal,11.6,2.0,3.75,2.875,0.247845,4.490891,4.744183,0.408981,0
258,Jota,Diogo,3.0,Liverpool,6.8,9.0,7.00,5.750,0.845588,4.190016,4.488235,0.660035,0
130,De Bruyne,Kevin,3.0,Man City,11.7,10.0,5.25,3.250,0.277778,4.693496,4.378987,0.374272,0
588,Ziyech,Hakim,3.0,Chelsea,8.3,2.0,7.50,4.000,0.481928,4.450288,4.154481,0.500540,1



position  4


,second_name,first_name,position,FIX_team,cost_today,LW_total_points,L4W_mean_points,L8W_mean_points,performance_per_cost,NW_forecast,N8W_mean_forecast,forecast_per_cost,in_team
88,Calvert-Lewin,Dominic,4.0,Everton,7.9,2.0,7.00,5.875,0.743671,6.700556,6.517554,0.825007,1
261,Kane,Harry,4.0,Spurs,10.9,2.0,5.25,7.875,0.722477,6.255753,6.240253,0.572500,1
575,Wilson,Callum,4.0,Newcastle,6.5,12.0,6.50,7.125,1.096154,6.165144,6.122476,0.941919,0
544,Vardy,Jamie,4.0,Leicester,10.2,5.0,6.25,6.375,0.625000,5.274121,5.239399,0.513667,0
594,de Jesus,Gabriel Fernando,4.0,Man City,9.3,5.0,4.00,2.000,0.215054,4.942175,4.469069,0.480545,0
565,Werner,Timo,4.0,Chelsea,9.5,2.0,5.25,5.375,0.565789,4.596739,4.365078,0.459482,1
561,Watkins,Ollie,4.0,Aston Villa,6.1,0.0,5.50,5.875,0.963115,4.768209,4.299743,0.704876,0
2,Adams,Che,4.0,Southampton,5.9,2.0,5.25,5.875,0.995763,4.604894,4.276981,0.724912,0
167,Firmino,Roberto,4.0,Liverpool,9.2,2.0,2.75,3.625,0.394022,4.065621,4.202911,0.456838,0
42,Bamford,Patrick,4.0,Leeds,6.1,2.0,3.25,5.125,0.840164,3.628871,3.982691,0.652900,0


# Update team

In [10]:
def update_team(current_team, player_list,transfer_out, transfer_in, to_file):
    
    current_team = current_team.loc[current_team['second_name'] != transfer_out]    
    
    new_player = player_list.loc[player_list['second_name'] == transfer_in][['first_name','second_name','position']].head(1)
    
    if transfer_in not in current_team['second_name']:
    
        current_team = current_team.append(new_player)  
        
    if to_file:
        current_team.drop_duplicates().sort_values('position').to_csv(Setup.teams_folder + str(date.today()) + '.csv', index = False)
    
    return current_team

In [11]:
update_team(current_team, player_GWs, 'Zaha', 'Jota', True)
current_team = load_most_recent_team()
current_team

Opening  data/teams/2020-12-03.csv


,first_name,second_name,position
0,Emiliano,MartÃ­nez,1.0
1,Alex,McCarthy,1.0
2,Timothy,Castagne,2.0
3,John,Egan,2.0
4,Tyrone,Mings,2.0
5,Luke,Thomas,2.0
6,Kurt,Zouma,2.0
7,Jack,Grealish,3.0
8,Heung-Min,Son,3.0
9,Georginio,Wijnaldum,3.0
